## Functions and parameters

In [93]:
import numpy as np

def first_x_sec(seconds, filepath, timecol):
    ''' param: seconds, int, filter out the first seconds of the game
        param: filepath, string, the filepath of the csv datafile
        param: timecol, int, the index of the column that contains time
    '''
    f = open(filepath)
    lines = f.readlines()
    lines = [ line.split(',') for line in lines ]
    if seconds is None:
        return np.array(lines[1:])
    else:
        return np.array(list(filter(lambda line: int(line[timecol])<=seconds, lines[1:])))

def read_columns(filepath, start_col, stop_col):
    f = open(filepath)
    lines = f.readlines()
    lines = [ line.strip().split(',')[start_col:stop_col] for line in lines[1:] ]
    return np.array(lines).astype(np.int)

def separate(matrix, col, index_function = lambda x: x):
    ''' This function will take a 2 diminsional numpy array and create a python list.
        each entry is a 2 dimensional numpy array. The entries in the list are separated 
        on the values in the given column
    '''
    max_index = np.max(map(index_function, matrix[:,col]))
    min_index = np.min(map(index_function, matrix[:,col]))
    list_length = max_index-min_index+1
    separated = [np.empty(shape=(0,matrix.shape[1])) for _ in range(list_length)]
    for i in range(list_length):
        separated[i] = np.array(filter(lambda x: index_function(x[col])==i, matrix))
    return separated

def save_array(array, filepath):
    print(array.shape)
    outfile = open(filepath, 'wb')
    np.save(outfile, array)
    outfile.close()


time_scope = 300
time_step = 60 # This is determined by data rather than by user here. Make sure how your data is separaed



## Match file contains more than just the label
# column 10 contains the True/False value of Radiant win

#purchases = first_x_sec(time_scope, 'dataset/purchase_log.csv', 1)





## Label data

In [94]:
labels = first_x_sec(None, 'dataset/match.csv', None)
labels = np.array([int(match[9] == 'True') for match in labels])
print(labels)
labels = np.array(filter(lambda (i,_): i not in too_short_matches, enumerate(labels)))
labels = labels[:,1]
labels =to_categorical(labels)
print(labels.shape)
print(labels)

[1 0 0 ..., 1 1 0]
(49996, 2)
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [95]:
save_array(labels, '/tmp/game_labels.npy')
labels = None # Clear memory

(49996, 2)


## Read player stats

In [96]:
# First 5 min: 5*60=300, col 2(1 in 0 indexing) is time.
player_time = first_x_sec(time_scope, 'dataset/player_time.csv', 1)
player_time = player_time.astype(np.float)
print(player_time.shape)




(299994, 32)


In [97]:
# Find what matches are too short
match_lengths = np.zeros((50000,1))
for log in player_time:
    match_lengths[log[0]] = log[1]
too_short_matches = []
for i in range(len(match_lengths)):
    if match_lengths[i] < time_scope:
        too_short_matches.append(i)
print(too_short_matches)
match_lengths = None

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[6213, 13914, 21228, 31438]


## Remove too short matches

In [98]:
player_time = np.array(filter(lambda log: log[0] not in too_short_matches, player_time))
print(player_time.shape)

(299976, 32)


## Separate data into time steps

In [99]:
# First we need to separate the data into time_steps so that we can normalize over the time steps
time_step_separated = separate(player_time, 1, index_function = lambda x: int(x/time_step))
player_time = None



## Normalize data in each time step over all matches

In [100]:
for i in range(len(time_step_separated)):
    max_value = np.max(time_step_separated[i][:,2:], axis=0)
    time_step_separated[i][:,2:] = time_step_separated[i][:,2:]/max_value
print(time_step_separated[1].shape)
print(time_step_separated[1])

(49996, 32)
[[  0.00000000e+00   6.00000000e+01   3.13891021e-01 ...,   5.00816993e-01
    1.25000000e-01   1.12107623e-01]
 [  1.00000000e+00   6.00000000e+01   7.59785111e-02 ...,   2.05065359e-01
    5.00000000e-01   2.40358744e-01]
 [  2.00000000e+00   6.00000000e+01   1.33537989e-01 ...,   8.08823529e-02
    0.00000000e+00   1.34529148e-02]
 ..., 
 [  4.99970000e+04   6.00000000e+01   1.08979279e-01 ...,   2.32026144e-01
    2.50000000e-01   2.06278027e-01]
 [  4.99980000e+04   6.00000000e+01   1.65003837e-01 ...,   8.16993464e-02
    0.00000000e+00   7.53363229e-02]
 [  4.99990000e+04   6.00000000e+01   7.59785111e-02 ...,   1.47875817e-01
    2.50000000e-01   7.35426009e-02]]


In [101]:
# Stick test that data is normalized
print(time_step_separated[3][3]) # timestep 3, match 3

[  3.00000000e+00   1.80000000e+02   2.34705656e-01   1.60000000e-01
   3.20872274e-01   1.40422078e-01   3.70370370e-02   4.22816115e-02
   1.89093762e-01   8.33333333e-02   3.71671991e-01   2.74174294e-01
   1.60000000e-01   2.55985267e-01   2.21856725e-01   2.30769231e-01
   6.15107914e-01   1.21813031e-01   7.69230769e-02   2.60069045e-01
   2.02346041e-01   2.40000000e-01   3.96303901e-01   1.12684195e-01
   3.70370370e-02   2.01164875e-01   1.33635334e-01   0.00000000e+00
   1.87090267e-01   4.95019489e-01   5.00000000e-01   4.59293395e-01]


## Change separation into matches

In [102]:
time_step_separated = np.array(time_step_separated)
print (time_step_separated.shape)
matches = np.swapaxes(time_step_separated, 0,1) #swap axes to have the data separated on matches now
time_step_separated = None

(6, 49996, 32)


In [103]:
print(matches.shape)
print(matches[0][1])

(49996, 6, 32)
[  0.00000000e+00   6.00000000e+01   3.13891021e-01   0.00000000e+00
   6.25620655e-02   1.03347889e-01   1.00000000e-01   9.55315871e-02
   1.33439237e-01   0.00000000e+00   1.30344108e-01   1.61812298e-01
   0.00000000e+00   1.62731872e-01   1.50387597e-01   1.25000000e-01
   1.16604478e-01   1.41119221e-01   2.50000000e-01   6.56996587e-02
   1.11200645e-01   1.25000000e-01   5.20134228e-02   2.58426966e-01
   7.50000000e-01   2.92743953e-01   7.48502994e-02   0.00000000e+00
   6.15507594e-02   5.00816993e-01   1.25000000e-01   1.12107623e-01]


In [104]:
matches = matches[:,:,2:]
print(matches.shape)

(49996, 6, 30)


## Save games without hero data

In [105]:
save_array(matches, '/tmp/game_data.npy')

(49996, 6, 30)


## Load in hero_data

In [67]:
from keras.utils.np_utils import to_categorical
hero_data = read_columns('dataset/players.csv', 0, 3)
hero_data = np.array(filter(lambda log: log[0] not in too_short_matches, hero_data))
hero_data = hero_data[:,2]
hero_rep = to_categorical(hero_data)
print(hero_rep.shape)
hero_data = None

(499960, 113)


## Create team hero representations

In [68]:
print(hero_rep[10].flatten().reshape((-1,1)).shape)
print(hero_rep.shape)
team_hero_rep = np.zeros(shape=(int(hero_rep.shape[0]/5),hero_rep.shape[1]))
print(team_hero_rep.shape)

for i in range(team_hero_rep.shape[0]):
    team_hero_rep[i] += np.sum(hero_rep[5*i:5*(i+1)], axis=0)
print(team_hero_rep.shape)
print(team_hero_rep[1])
hero_rep = None

(113, 1)
(499960, 113)
(99992, 113)
(99992, 113)
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.]


In [69]:
## reshape so that first dimension is number of matches and second is the 2  team vectors concatenated
team_hero_rep = team_hero_rep.reshape((matches.shape[0],team_hero_rep.shape[1]*2))
print(team_hero_rep.shape)

(49996, 226)


## Add the team hero data to the match vectors

In [70]:
print(matches.shape)
## Repeat info for all time steps
team_hero_rep = np.repeat(team_hero_rep, matches.shape[1], axis=1)
## Reshape into time steps
team_hero_rep = team_hero_rep.reshape((team_hero_rep.shape[0], team_hero_rep.shape[1]//matches.shape[1], matches.shape[1]))
## Swap the axes to match with matches
team_hero_rep = np.swapaxes(team_hero_rep, 1,2)
print(team_hero_rep.shape)
matches = np.concatenate((matches, team_hero_rep), axis=2)
print(matches.shape)

(49996, 6, 30)
(49996, 6, 226)
(49996, 6, 256)


## Save games with hero data

In [ ]:
save_array(matches, '/tmp/game_data.npy')

In [71]:
#import math
#print(purchases.shape)
#purchases = purchases.astype(np.int)

    
#for log in purchases:
#    log[1] = math.ceil(max(0, log[1]) / 60) * 60
 



In [72]:
#in any game, any player for one timestep, what is maximum number of items?
#matches = [np.empty(shape=(4,0), dtype=np.int)]*50000
#for x in purchases:
#    matches[x[3]] = np.concatenate((matches[x[3]], x.reshape(4,1)), axis=1)

# separate on time
#print(matches[1].T)
#for i, match in enumerate(matches):
#    separated = [np.empty(shape=(4,0), dtype=np.int)]*6 # 6 time steps [0 60 120 180 240 300]
#    for x in match.T:
#        time = int(x[1]/60)
#        separated[time] = np.concatenate((separated[time], x.reshape(4,1)), axis=1)
#    matches[i] = separated

#Separate on player
#for match in matches:
#    for i, time_step in enumerate(match):
#        #problem is that this vector is the same for all players. Need 10 copies
#        separated = [np.zeros(shape=(218,1), dtype=np.int) for j in range(10)]*10 # 10 players and 218 is maximum item id in dataset
        
#        for x in time_step.T:
#            player = x[2] if x[2]<5 else x[2] - 123 #make players numbered 0 to 9
#            separated[player][x[0]-1] += 1
#        match[i] = separated
 

In [73]:
#print (matches[15][0][2].shape) # first match, first timestep 0 and first player purchase log